### Dataset

The following is a display of supermarket transaction data throughout 2019, complete with an explanation for each column.
* `order_id` : ID dari order/transaksi, 1 transaksi bisa terdiri dari beberapa produk, tetapi hanya dilakukan oleh 1 customer
* `order_date` : tanggal terjadinya transaksi
* `customer_id` : ID dari pembeli, bisa jadi dalam satu hari, 1 customer melakukan transaksi beberapa kali
* `city` : kota tempat toko terjadinya transaksi
* `province` : provinsi (berdasarkan city)
* `product_id` : ID dari suatu product yang dibeli
* `brand` : brand/merk dari product. Suatu product yang sama pasti memiliki brand yang sama
* `quantity` : Kuantitas/banyaknya product yang dibeli
* `item_price` : Harga dari 1 product (dalam Rupiah). Suatu product yang sama, bisa jadi memiliki harga yang berbeda saat dibeli


location data:
https://storage.googleapis.com/dqlab-dataset/retail_raw_reduced.csv

### Import Library

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import html
from dash import dcc
from dash import Input
from dash import Output

### Data Preparation

In [2]:
df = pd.read_csv('https://storage.googleapis.com/dqlab-dataset/retail_raw_reduced.csv')

In [3]:
df.head()

,order_id,order_date,customer_id,city,province,product_id,brand,quantity,item_price
0,1703458,2019-10-17,14004,Jakarta Selatan,DKI Jakarta,P1910,BRAND_J,10,740000
1,1706815,2019-10-24,17220,Jakarta Selatan,DKI Jakarta,P2934,BRAND_R,2,604000
2,1710718,2019-11-03,16518,Jakarta Utara,DKI Jakarta,P0908,BRAND_C,8,1045000
3,1683592,2019-08-19,16364,Jakarta Barat,DKI Jakarta,P0128,BRAND_A,4,205000
4,1702573,2019-10-16,15696,Jakarta Timur,DKI Jakarta,P2968,BRAND_R,2,4475000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   order_id     5000 non-null   int64 
 1   order_date   5000 non-null   object
 2   customer_id  5000 non-null   int64 
 3   city         5000 non-null   object
 4   province     5000 non-null   object
 5   product_id   5000 non-null   object
 6   brand        5000 non-null   object
 7   quantity     5000 non-null   int64 
 8   item_price   5000 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 351.7+ KB


### Exploratory Data Analysis

Grouping data for GMV based on month by month

In [5]:
df['gmv'] = df['item_price'] * df['quantity']
df.head()
 

,order_id,order_date,customer_id,city,province,product_id,brand,quantity,item_price,gmv
0,1703458,2019-10-17,14004,Jakarta Selatan,DKI Jakarta,P1910,BRAND_J,10,740000,7400000
1,1706815,2019-10-24,17220,Jakarta Selatan,DKI Jakarta,P2934,BRAND_R,2,604000,1208000
2,1710718,2019-11-03,16518,Jakarta Utara,DKI Jakarta,P0908,BRAND_C,8,1045000,8360000
3,1683592,2019-08-19,16364,Jakarta Barat,DKI Jakarta,P0128,BRAND_A,4,205000,820000
4,1702573,2019-10-16,15696,Jakarta Timur,DKI Jakarta,P2968,BRAND_R,2,4475000,8950000


In [6]:
df['order_month'] = df['order_date'].str.slice(0, 7)
df.head()

,order_id,order_date,customer_id,city,province,product_id,brand,quantity,item_price,gmv,order_month
0,1703458,2019-10-17,14004,Jakarta Selatan,DKI Jakarta,P1910,BRAND_J,10,740000,7400000,2019-10
1,1706815,2019-10-24,17220,Jakarta Selatan,DKI Jakarta,P2934,BRAND_R,2,604000,1208000,2019-10
2,1710718,2019-11-03,16518,Jakarta Utara,DKI Jakarta,P0908,BRAND_C,8,1045000,8360000,2019-11
3,1683592,2019-08-19,16364,Jakarta Barat,DKI Jakarta,P0128,BRAND_A,4,205000,820000,2019-08
4,1702573,2019-10-16,15696,Jakarta Timur,DKI Jakarta,P2968,BRAND_R,2,4475000,8950000,2019-10


In [7]:
df_grouped_gmv = df.groupby('province')['gmv'].sum().reset_index()
df_grouped_gmv.sort_values(by='gmv', ascending=False)

,province,gmv
2,DKI Jakarta,14902763000
3,Jawa Barat,6128149000
4,Jawa Tengah,3485884000
6,Yogyakarta,3111241000
5,Jawa Timur,2649486000
1,Banten,1495632000
0,Bali,1201212000


In [8]:
df_montly_agg_by_city = df[df['province'] == 'DKI Jakarta']
df_montly_agg_by_city = df_montly_agg_by_city.groupby(['order_month', 'city'])['gmv'].sum().reset_index()
df_montly_agg_by_city

,order_month,city,gmv
0,2019-07,Jakarta Barat,241935000
1,2019-07,Jakarta Pusat,646077000
2,2019-07,Jakarta Selatan,578279000
3,2019-07,Jakarta Timur,167781000
4,2019-07,Jakarta Utara,317799000
5,2019-08,Jakarta Barat,236758000
6,2019-08,Jakarta Pusat,603726000
7,2019-08,Jakarta Selatan,802238000
8,2019-08,Jakarta Timur,148771000
9,2019-08,Jakarta Utara,222900000


In [9]:
# Create multi line chart
x = df_montly_agg_by_city['order_month']
y = df_montly_agg_by_city['gmv']

fig = px.line(
    df_montly_agg_by_city, 
    x=x, 
    y=y,
    color='city'
)

fig.update_layout(
    title='Total Gross Merchandise Value (GMV) by City in DKI Jakarta',
    title_font=dict(color='#508ef2'),
    title_x = 0.5,
    xaxis_title = 'Order Month',
    yaxis_title = 'GMV',
    paper_bgcolor="#000000",
    legend_font_color="#FFFFFF"
)

fig.update_xaxes(
    title_font=dict(
        size=15, 
        family='arial', 
        color='white'
    ),
    tickangle = 45,
    color='#b5b1b1'
)

fig.update_yaxes(
    title_font=dict(
        size=15, 
        family='arial', 
        color='white'
    ),
    color='#b5b1b1'
)

fig.show()

### Dashboarding with Dash

In [10]:
df_montly_agg_by_city.head()

,order_month,city,gmv
0,2019-07,Jakarta Barat,241935000
1,2019-07,Jakarta Pusat,646077000
2,2019-07,Jakarta Selatan,578279000
3,2019-07,Jakarta Timur,167781000
4,2019-07,Jakarta Utara,317799000


In [11]:
available_cities = df_montly_agg_by_city['city'].unique()
available_cities

array(['Jakarta Barat', 'Jakarta Pusat', 'Jakarta Selatan',
       'Jakarta Timur', 'Jakarta Utara'], dtype=object)

In [12]:
# Create a dash app
app = dash.Dash()

# Dark theme
dark_theme = {
    "main-background": "#000000",
    "header-text": "#ff7575",
    "sub-text": "#ffd175",
}

# Design a layout
app.layout = html.Div(
    children=[
        html.H1(
            'Dashboard',
            style={
                'color':'#508ef2',
                'font-size':40,
                'text-align':'center'
            }
        ),
        dcc.Dropdown(
            id='my-dropdown',
            options=[{
                'label':k,
                'value':k
            } for k in available_cities],
            value=['Jakarta Selatan', 'Jakarta Pusat'],
            multi=True
        ),
        html.Hr(),
        dcc.Graph(id='display-value')
    ],
    style={
        'backgroundColor':dark_theme["main-background"]
    }
)

# Callback
@app.callback(
    Output(
        'display-value',
        component_property='figure'
    ),
    Input(
        'my-dropdown',
        component_property='value'
    )
)


def update_layout(value):
    # Create plot
    my_data = df_montly_agg_by_city[df_montly_agg_by_city['city'].isin(value)]
    fig_dash = px.line(
        my_data, 
        x='order_month', 
        y='gmv', 
        color='city', 
        template='ggplot2'
    )

    # Modify title
    fig_dash.update_layout(
        title='Total Gross Merchandise Value (GMV) by City in DKI Jakarta',
        title_font=dict(color='#508ef2'),
        title_x = 0.5,
        xaxis_title = 'Order Month',
        yaxis_title = 'GMV',
        paper_bgcolor="#000000",
        legend_font_color="#FFFFFF"
    )

    # Update X & Y Label
    fig_dash.update_xaxes(
        title_font=dict(
            size=15, 
            family='arial', 
            color='white'
        ),
        tickangle = 45,
        color='#b5b1b1'
    )
    fig_dash.update_yaxes(
        title_font=dict(
            size=15, 
            family='arial', 
            color='white'
        ),
        color='#b5b1b1'
    )
    return fig_dash

# Run app
if __name__ == '__main__':
    app.run_server(host='localhost', port=8005)

Dash is running on http://localhost:8005/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8005
Press CTRL+C to quit
127.0.0.1 - - [23/Oct/2022 22:07:38] "GET /_favicon.ico?v=2.6.2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2022 22:07:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2022 22:07:41] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2022 22:07:41] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2022 22:07:41] "GET /_favicon.ico?v=2.6.2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2022 22:07:41] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Oct/2022 22:07:41] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Oct/2022 22:07:41] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Oct/2022 22:07:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2022 22:07:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2022 22:07:49] "POST /_dash-update-component HTTP/1.1" 200 -
